In [1]:
import re
from string import punctuation
import nltk
import string
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [2]:
def normalize(text):
    processed_text =  re.sub(f"[{re.escape(punctuation)}]", "", text)
    processed_text = " ".join(processed_text.split())
    return processed_text

def lemmatize(processed_text):
    wordnet_lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    required_words = [wordnet_lemmatizer.lemmatize(x, 'v') for x in  tokens]
    sentence_with_lemmnatized_word = " ".join(required_words)
    return sentence_with_lemmnatized_word

def process_text(text):
    text = normalize(text)
    text = lemmatize(text)
    return text

In [3]:
#import nltk
#from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

def create_pos_tags(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent




In [4]:
import stanza
#stanza.download ('en')

def get_named_entities(text):
    nlp = stanza.Pipeline ('en', download_method=stanza.DownloadMethod.NONE)
    results = nlp (text)
    return results.entities

In [5]:


# Bag of Words
prop_list = [['has_director','direct','directed by','director'], ['has_cast','cast'],['duration', 'length'],['brand','produce','manufactured by','manufacture',"make","made"],
             ["Price",'price'],['Available_Colors','color','colour'],['Material'],['Contact_number','telephone'],['Joined_Date','join'],['Shipping_Address','shipping', 'billing','bil']]
class_list = [['movie', 'film', 'work_of_art'],['People', 'person'],['language'],['furniture', 'product']]

In [6]:
def map_property(word, prop_list):
    mapped_property = ""
    index = [[i, prop.index(word)]
             for i, prop in enumerate(prop_list)
             if word in prop]
    if len(index) > 0:
        mapped_property = prop_list[index[0][0]][0]
    return mapped_property

def map_class(word, class_list):
    mapped_class = ""
    index = [[i, cls.index(word)]
             for i, cls in enumerate(class_list)
             if word in cls]
    if len(index) > 0:
        mapped_class = class_list[index[0][0]][0]
    return mapped_class

In [7]:

text = "what is the price of Chesterfield_Sofa"
#text = "what are the colors in Fabric_Dining_Chair"
#text= "What is the joined date of Tharani?"
#text = "what is the telephone number of James"
#text = "All of bedroom_Furniture manufactured by ModernVases"


In [8]:
processed_text = process_text(text)
tags = create_pos_tags(processed_text)
tags

[('what', 'WP'),
 ('be', 'VB'),
 ('the', 'DT'),
 ('price', 'NN'),
 ('of', 'IN'),
 ('ChesterfieldSofa', 'NNP')]

In [9]:
def map_property(word, prop_list):
    mapped_property = ""
    index = [[i, prop.index(word)]
             for i, prop in enumerate(prop_list)
             if word in prop]
    if len(index) > 0:
        mapped_property = prop_list[index[0][0]][0]
    return mapped_property

def map_class(word, class_list):
    mapped_class = ""
    index = [[i, cls.index(word)]
             for i, cls in enumerate(class_list)
             if word in cls]
    if len(index) > 0:
        mapped_class = class_list[index[0][0]][0]
    return mapped_class

In [10]:
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#!pip install pattern
from pattern.text.en import pluralize

tagged_chuncks = nltk.ne_chunk(tags) 
# Iterate over the named entities and print their labels 
classes = []
properties = []
individuals = {}

for entity in tagged_chuncks: 
    if hasattr(entity, "label"): 
         print("check entity: ",entity)
    else:
        if entity[1] == "NNS":
            mapped_class = map_class(pluralize(entity[0]), class_list)
            if mapped_class!="":
                classes.append(mapped_class.pluralize())
        elif entity[1] == "VBP":
            mapped_property = map_property(entity[0], prop_list)
            if mapped_property!="": 
                properties.append(mapped_property)
            
        elif entity[1] == "NN":
            mapped_property = map_property(entity[0], prop_list)
            mapped_class = map_class(pluralize(entity[0]), class_list)
            if mapped_property!="": 
                properties.append(mapped_property)
            elif mapped_class!="":
                classes.append(mapped_class.capitalize())

######################## CREATE INDIVIDUALS ##############################
named_entities = get_named_entities(text)

for e in named_entities: 
    print(e)
    mapped_ind_class = map_class(pluralize(e.type.lower()), class_list)
    if mapped_ind_class!="":
        individuals[e.text] = mapped_ind_class.capitalize()
    else:
        individuals[e.text] = "not_mapped"
        #classes.append(mapped_ind_class.capitalize())

print("Classes: ",classes)
print("Properties: ",properties)
print("Individuals: ",individuals)

2023-11-20 17:48:46 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| pos          | combined_charlm     |
| lemma        | combined_nocharlm   |
| constituency | ptb3-revised_charlm |
| depparse     | combined_charlm     |
| sentiment    | sstplus             |
| ner          | ontonotes_charlm    |

2023-11-20 17:48:46 INFO: Using device: cpu
2023-11-20 17:48:46 INFO: Loading: tokenize


check entity:  (ORGANIZATION ChesterfieldSofa/NNP)


2023-11-20 17:48:46 INFO: Loading: pos
2023-11-20 17:48:46 INFO: Loading: lemma
2023-11-20 17:48:46 INFO: Loading: constituency
2023-11-20 17:48:47 INFO: Loading: depparse
2023-11-20 17:48:47 INFO: Loading: sentiment
2023-11-20 17:48:47 INFO: Loading: ner
2023-11-20 17:48:47 INFO: Done loading processors!


{
  "text": "Chesterfield_Sofa",
  "type": "ORG",
  "start_char": 21,
  "end_char": 38
}
Classes:  []
Properties:  ['Price']
Individuals:  {'Chesterfield_Sofa': 'not_mapped'}


In [11]:
from SPARQLWrapper import SPARQLWrapper2
sparql = SPARQLWrapper2("http://localhost:3030/FurnitureShopManagment/sparql")
c_triple = ""
p_triple = ""
i_triple = ""


if len(classes)>0:
    c_triple = "{?y a movie:"+classes[0]+"}"
    if(len(individuals)>0):
        c_triple = c_triple + "{?x (movie:|!movie:)|^(movie:|!movie:)* ?y}"
if len(properties)>0:
    p_triple = "{?x movie:"+properties[0]+"|^movie:"+properties[0]+" ?y}"

    
if  len(individuals)>0:
    first_key = next(iter(individuals))
    if(individuals[first_key]=="not_mapped"):
        if(len(properties)==0):
            i_triple = "{?x a ?y} {?x movie:name ?name} FILTER(?name='"+first_key+"')"            
        else:
            i_triple = " {?x movie:name ?name} FILTER(?name='"+first_key+"')"
    else:
         i_triple = "{?x a movie:"+individuals[first_key]+"} {?x movie:name ?name} FILTER(?name='"+first_key+"')"
            
query_start =      """
                PREFIX movie: <http://www.semanticweb.org/chand/ShopManagementSystem#>
                SELECT  ?y
                WHERE{
                """
query_end =    "}"

query = query_start+c_triple+p_triple+i_triple+query_end
print(query)
sparql.setQuery(query)
result = sparql.query().bindings


                PREFIX movie: <http://www.semanticweb.org/chand/ShopManagementSystem#>
                SELECT  ?y
                WHERE{
                {?x movie:Price|^movie:Price ?y} {?x movie:name ?name} FILTER(?name='Chesterfield_Sofa')}


In [12]:
for x in result:
    print(x["y"].value)

1499.99
